# 4. Run Cytoplasmic Detection and Quantification 

gg-napari-env

In [13]:
from napari_czifile2 import napari_get_reader
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm
import sys 
import os 
from concurrent.futures import ThreadPoolExecutor, as_completed
from scipy.ndimage import binary_dilation, generate_binary_structure
import xml.etree.ElementTree as ET
import warnings
warnings.filterwarnings("ignore")

In [ ]:
raw_data_dirs = os.listdir('../../../RNA-FISH-raw-data/')
raw_data_dirs

In [ ]:
input = 'T79'
input = [d for d in raw_data_dirs if input in d][0]
print(f'Using {input} as input directory')
input_dir = f'../raw-data/{input}/'
assert os.path.exists(input_dir), 'Input directory does not exist'
czi_files = [f for f in os.listdir(input_dir) if f.endswith('.czi')]
print(f"Found {len(czi_files)} czi files in {input_dir}")
print(czi_files)

Using 20250328 1 P14 T79-intergenic-b2-647 T79-exonic-b1-546 DAPI as input directory
Found 5 czi files in ../raw-data/20250328 1 P14 T79-intergenic-b2-647 T79-exonic-b1-546 DAPI/
['20250328 1 T79 sample 1.czi', '20250328 1 T79 sample 5.czi', '20250328 1 T79 sample 2.czi', '20250328 1 T79 sample 3.czi', '20250328 1 T79 sample 4.czi']


In [15]:
def normalize_stack_by_dapi(image, dapi_channel=0, method='mean', eps=1e-8):
    """
    Normalize all non-DAPI channels in a z-stack using DAPI intensity as reference.

    Parameters:
    - image: np.ndarray, shape (Z, C, X, Y)
    - dapi_channel: int, index of the DAPI channel (in the C dimension)
    - method: str, 'mean', 'median', or 'max' for calculating DAPI intensity per z-slice
    - eps: small value to avoid division by zero

    Returns:
    - corrected: np.ndarray, shape (Z, C, X, Y)
    """
    
    z, c, x, y = image.shape
    corrected = np.zeros_like(image, dtype=np.uint8)

    # Extract DAPI channel across z
    dapi_stack = image[:, dapi_channel, :, :]

    # Compute per-slice intensity
    if method == 'mean':
        dapi_intensity = dapi_stack.mean(axis=(1, 2))
    elif method == 'median':
        dapi_intensity = np.median(dapi_stack, axis=(1, 2))
    elif method == 'max':
        dapi_intensity = dapi_stack.max(axis=(1, 2))
    else:
        raise ValueError("method must be 'mean', 'median', or 'max'")

    # Normalize: scale each slice by the ratio of max intensity to current slice intensity
    dapi_ref = np.max(dapi_intensity)
    correction_factors = dapi_ref / (dapi_intensity + eps)

    for z_idx in range(z):
        for ch in range(c):
            if ch == dapi_channel:
                # Leave DAPI channel unchanged
                corrected[z_idx, ch] = image[z_idx, ch]
            else:
                corrected[z_idx, ch] = (image[z_idx, ch] * correction_factors[z_idx])

    # Clip values to ensure they are within valid range for uint8
    corrected = np.clip(corrected, 0, 255).astype(np.uint8)
    
    return corrected

In [16]:
# Function to process each slice
def get_slice_borders(z, all_rois, results, cell_borders, pixels_to_dilate, struct_elem):
    rois_slice = all_rois[z, :, :]
    cell_borders_slice = cell_borders[z, :, :]
    cells_slice = results[results['z'] == z]
    
    for i, cell in cells_slice.iterrows():
        # Get specific cell mask 
        cell_mask = rois_slice == cell['z_id']
        # Dilate region 
        dilated_mask = binary_dilation(cell_mask, structure=struct_elem, iterations=pixels_to_dilate)
        # Subtract the two regions using bitwise XOR
        cell_border = dilated_mask ^ cell_mask
        # create a temporary mask to store the cell border
        temp = cell_border 
        # Remove areas in cell_border where cell_borders is non-zero 
        cell_border[cell_borders_slice > 0] = 0
        # Remove areas in cell_border where the nuclei are non-zero 
        cell_border[rois_slice > 0] = 0
        # Remove areas in cell_borders where cell_border is non-zero
        cell_borders_slice[temp > 0] = 0
        # Save new areas to the cell_borders array 
        cell_borders_slice[cell_border] = cell['z_id']
    
    # Return the updated slice to be stored back
    return z, cell_borders_slice

# Parallelize processing of Z slices
def get_all_slice_borders(all_rois, results, cell_borders, pixels_to_dilate, struct_elem, n_threads):
    with ThreadPoolExecutor(max_workers=n_threads) as executor:
        # Create a list of futures to process each slice in parallel
        futures = [executor.submit(get_slice_borders, z, all_rois, results, cell_borders, pixels_to_dilate, struct_elem) for z in range(all_rois.shape[0])]
        
        # Wait for all futures to complete and store the results
        for future in as_completed(futures):
            z, updated_cell_borders_slice = future.result()
            # After processing, store the updated slice back into cell_borders
            cell_borders[z, :, :] = updated_cell_borders_slice
    
    return cell_borders

In [17]:
from skimage.filters import gaussian

def subtract_background_gaussian_uint8(slice_img, sigma=100):
    """
    Subtract background fluorescence from each channel in a single z-slice using Gaussian filtering,
    and return the result as an 8-bit image.

    Parameters
    ----------
    slice_img : numpy.ndarray
        A 3D numpy array of shape (channels, height, width) with dtype=np.uint8.
    sigma : float, optional
        Standard deviation for Gaussian filter to estimate the background.
        Adjust based on the spatial scale of the background variation; default is 100.

    Returns
    -------
    bg_subtracted : numpy.ndarray
        Background-subtracted image with dtype=np.uint8, same shape as slice_img.
    """
    # Prepare an output array of the same shape with dtype uint8
    bg_subtracted = np.empty_like(slice_img, dtype=np.uint8)
    
    # Process each channel independently
    for ch in range(slice_img.shape[0]):
        # Estimate the background using a Gaussian filter
        background = gaussian(slice_img[ch], sigma=sigma, preserve_range=True)
        # Subtract the background in float32 to capture potential negative values
        subtracted = slice_img[ch].astype(np.float32) - background.astype(np.float32)
        # Clip the values to ensure they fall within the valid 8-bit range and convert back to uint8
        subtracted_uint8 = np.clip(subtracted, 0, 255).astype(np.uint8)
        bg_subtracted[ch] = subtracted_uint8
    
    return bg_subtracted

In [ ]:
def threshold_triangle_min_area(slice_img, min_area=12):
    """
    Apply Triangle threshold (automatic) on each channel in a single z-slice,
    and return the image with only the thresholded signal remaining,
    filtered by minimum area.

    Parameters
    ----------
    slice_img : numpy.ndarray
        A 3D numpy array of shape (channels, height, width) with dtype=np.uint8.
    min_area : int
        Minimum area (in pixels) for connected components to be kept.

    Returns
    -------
    thresholded : numpy.ndarray
        Thresholded image with dtype=np.uint8, same shape as slice_img.
        For each channel, pixels with values below the Triangle threshold are set to 0.
        Only components with area > min_area are retained.
    """
    import numpy as np
    from skimage.filters import threshold_triangle
    from skimage.morphology import remove_small_objects

    # Prepare an output array of the same shape with dtype uint8
    thresholded = np.zeros_like(slice_img, dtype=np.uint8)

    # Process each channel independently
    for ch in range(slice_img.shape[0]):
        # Compute the threshold using the Triangle method for the current channel
        thresh = threshold_triangle(slice_img[ch])

        # Threshold image
        binary_mask = slice_img[ch] >= thresh

        # Remove small objects based on min_area
        cleaned_mask = remove_small_objects(binary_mask, min_size=min_area)

        # Apply cleaned mask to original image
        thresholded[ch] = np.where(cleaned_mask, slice_img[ch], 0)

    return thresholded

In [21]:
# Function to process each slice
def quantify_border(z, image_data, results, cell_borders, channels):
    image_slice = image_data[z, :, :, :]
    cell_borders_slice = cell_borders[z, :, :]
    cells_slice = results[results['z'] == z]

    # Background subtraction 
    image_slice = subtract_background_gaussian_uint8(image_slice, sigma=100)

    # Thresholding
    #image_slice = threshold_triangle_uint8(image_slice)
    image_slice = threshold_triangle_min_area(image_slice, min_area=12)
    
    # Create list to store results for each cell
    cell_borders_results = [] 
    for i, cell in cells_slice.iterrows():
        cell_border_result = {} 
        cell_border_result['z'] = z
        cell_border_result['z_id'] = cell['z_id']
        cell_border_result['border_pxls'] = np.sum(cell_borders_slice == cell['z_id'])
        for channel_index, channel_name in channels:
            # Calculate mean intensity in the border area for each channel
            cell_border_result[channel_name + "-cyto-mean"] = np.mean(image_slice[channel_index, cell_borders_slice == cell['z_id']])
            cell_border_result[channel_name + "-cyto-median"] = np.median(image_slice[channel_index, cell_borders_slice == cell['z_id']])
            cell_border_result[channel_name + "-cyto-sum"] = np.sum(image_slice[channel_index, cell_borders_slice == cell['z_id']])
        cell_borders_results.append(cell_border_result)
    
    # Convert the list of results to DataFrame
    cell_borders_results = pd.DataFrame(cell_borders_results)
        
    return cell_borders_results

# Parallelize processing of Z slices
def parallelize_quantify_borders(image_data, results, cell_borders, channels, n_threads):
    # Create a list to collect results for each Z slice
    all_cell_border_results = []
    
    with ThreadPoolExecutor(max_workers=n_threads) as executor:
        # Create a list of futures to process each slice in parallel
        futures = [executor.submit(quantify_border, z, image_data, results, cell_borders, channels) for z in range(image_data.shape[0])]
        
        # Wait for all futures to complete and collect the results
        for future in as_completed(futures):
            cell_borders_results = future.result()
            all_cell_border_results.append(cell_borders_results)
    
    # Concatenate all results into a single DataFrame
    all_cell_border_results = pd.concat(all_cell_border_results, axis=0, ignore_index=True)
    
    # After processing all slices, merge the results with the original `results` DataFrame
    results = pd.merge(results, all_cell_border_results, on=['z', 'z_id'], how='left')
    
    return results

In [22]:
# Pixel of 3 = 270nm. We think from EM images the cytoplasm is 285nm wide 
pixels_to_dilate = 3
n_threads = 50 

In [12]:
for f in czi_files[:]:
    print(f"Processing {f}")
    file_path = os.path.join(input_dir, f)
    reader = napari_get_reader(file_path)
    if reader is not None:
        layer_data = reader(file_path)
        image_data, metadata, layer_type = layer_data[0]

        # Remove singleton dimension 
        image_data = np.squeeze(image_data)  
        print("Metadata:", metadata)
        print("Image shape:", image_data.shape)  

        # Load ROIs 
        all_rois_path = f'../results/{input}/{f.replace(".czi", "_rois.npy")}'
        all_rois = np.load(all_rois_path)
        print(f"Loaded {all_rois_path}")

        # Load results 
        results_path = f'../results/{input}/{f.replace(".czi", "_nuclei.csv")}'
        results_image = pd.read_csv(results_path)
        results_image.columns = results_image.columns.str.split('-T').str[0]
        print(f"Loaded {results_path}")

        dapi_index = metadata['name'].index([name for name in metadata['name'] if 'DAPI' in name][0])
        print(f"DAPI channel index: {dapi_index}")

        non_dapi_channels = [(i, metadata['name'][i].split('-T')[0]) for i in range(len(metadata['name'])) if i != dapi_index]
        print(f"Non-DAPI channels: {non_dapi_channels}")

        image_data = normalize_stack_by_dapi(image_data, dapi_channel=dapi_index, method='mean')
        print("Normalized image data using DAPI intensity.")
        
        # Get the cell borders 
        cell_borders = np.zeros_like(all_rois)
        struct_elem = generate_binary_structure(2, 1)
        cell_borders = get_all_slice_borders(all_rois, results_image, cell_borders, pixels_to_dilate, struct_elem, n_threads=n_threads)

        # Save the cell borders
        cell_borders_path = f'../results/{input}/{f.replace(".czi", "_cell_borders.npy")}'
        print(f"Saving cell borders to {cell_borders_path}")
        np.save(cell_borders_path, cell_borders)

        # Quantify 
        results_image = parallelize_quantify_borders(image_data, results_image, cell_borders, non_dapi_channels, n_threads=n_threads)

        # Save the results 
        results_image_path = f'../results/{input}/{f.replace(".czi", "_with_borders.csv")}'
        print(f"Saving results to {results_image_path}")
        results_image.to_csv(results_image_path, index=False)
        

Processing 20250328 1 T79 sample 1.czi
Metadata: {'rgb': False, 'channel_axis': 2, 'translate': (0.0, 0.0, 29259.974395751953, 31219.198837280273), 'scale': (1.0, 1.0, 0.0974884033203125, 0.0974884033203125), 'contrast_limits': None, 'name': ['AF546-T1', 'DAPI-T2', 'AF647-T2']}
Image shape: (79, 3, 2048, 2048)
Loaded ../results/20250328 1 P14 T79-intergenic-b2-647 T79-exonic-b1-546 DAPI/20250328 1 T79 sample 1_rois.npy
Loaded ../results/20250328 1 P14 T79-intergenic-b2-647 T79-exonic-b1-546 DAPI/20250328 1 T79 sample 1_nuclei.csv
DAPI channel index: 1
Non-DAPI channels: [(0, 'AF546'), (2, 'AF647')]
Normalized image data using DAPI intensity.
Saving cell borders to ../results/20250328 1 P14 T79-intergenic-b2-647 T79-exonic-b1-546 DAPI/20250328 1 T79 sample 1_cell_borders.npy
Saving results to ../results/20250328 1 P14 T79-intergenic-b2-647 T79-exonic-b1-546 DAPI/20250328 1 T79 sample 1_with_borders.csv
Processing 20250328 1 T79 sample 5.czi
Metadata: {'rgb': False, 'channel_axis': 2, 't

## Run on all images

In [ ]:
raw_data_dirs = os.listdir('../../../RNA-FISH-raw-data/')
raw_data_p14 = [d for d in raw_data_dirs if '14' in d]
raw_data_p14

['20250307 B1 P14 U34-B3-546 Chymotrypsin-B2-647 DAPI',
 '20250328 1 P14 T79-intergenic-b2-647 T79-exonic-b1-546 DAPI',
 '20250328 5 P14 LOC603-b3-488 9E108-b1-546 9E116-b2-647 DAPI',
 '20250325 5 p14 g1-b1-546 lnc7-b2-647 dapi',
 '20250328 2 P14 R2-b3-488 Q1-b1-546 Lnc6-b2-647 DAPI',
 '20250328 4 P14 9E129-b3-488 LOC104-b1-546 9E116-b2-647 dapi',
 '20250325 4 p14 u34-b3-488 lnc4-b1-546 u21-b5-647 dapi',
 '20250328 3 P14 Lnc3-b3-488 L16-b2-594 Lnc2-b5-647 DAPI']

In [24]:
for input in raw_data_p14: 

    input_dir = f'../raw-data/{input}/'
    assert os.path.exists(input_dir), 'Input directory does not exist'
    czi_files = [f for f in os.listdir(input_dir) if f.endswith('.czi')]
    print(f"Found {len(czi_files)} czi files in {input_dir}")
    print(czi_files)

    results_dir = f'../results/{input}'
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
        
    for f in czi_files[:]:
        print(f"Processing {f}")
        file_path = os.path.join(input_dir, f)
        reader = napari_get_reader(file_path)
        if reader is not None:
            layer_data = reader(file_path)
            image_data, metadata, layer_type = layer_data[0]

            # Remove singleton dimension 
            image_data = np.squeeze(image_data)  
            print("Metadata:", metadata)
            print("Image shape:", image_data.shape)  

            # Load ROIs 
            all_rois_path = f'../results/{input}/{f.replace(".czi", "_rois.npy")}'
            all_rois = np.load(all_rois_path)
            print(f"Loaded {all_rois_path}")

            # Load results 
            results_path = f'../results/{input}/{f.replace(".czi", "_nuclei.csv")}'
            results_image = pd.read_csv(results_path)
            results_image.columns = results_image.columns.str.split('-T').str[0]
            print(f"Loaded {results_path}")

            dapi_index = metadata['name'].index([name for name in metadata['name'] if 'DAPI' in name][0])
            print(f"DAPI channel index: {dapi_index}")

            non_dapi_channels = [(i, metadata['name'][i].split('-T')[0]) for i in range(len(metadata['name'])) if i != dapi_index]
            print(f"Non-DAPI channels: {non_dapi_channels}")

            image_data = normalize_stack_by_dapi(image_data, dapi_channel=dapi_index, method='mean')
            print("Normalized image data using DAPI intensity.")
            
            # Get the cell borders 
            cell_borders = np.zeros_like(all_rois)
            struct_elem = generate_binary_structure(2, 1)
            cell_borders = get_all_slice_borders(all_rois, results_image, cell_borders, pixels_to_dilate, struct_elem, n_threads=n_threads)

            # Save the cell borders
            cell_borders_path = f'../results/{input}/{f.replace(".czi", "_cell_borders.npy")}'
            print(f"Saving cell borders to {cell_borders_path}")
            np.save(cell_borders_path, cell_borders)

            # Quantify 
            results_image = parallelize_quantify_borders(image_data, results_image, cell_borders, non_dapi_channels, n_threads=n_threads)

            # Save the results 
            results_image_path = f'../results/{input}/{f.replace(".czi", "_with_borders.csv")}'
            print(f"Saving results to {results_image_path}")
            results_image.to_csv(results_image_path, index=False)
            

Found 5 czi files in ../raw-data/20250307 B1 P14 U34-B3-546 Chymotrypsin-B2-647 DAPI/
['20250307 B1 Sample 5 Stack.czi', '20250307 B1 Sample 4 Stack.czi', '20250307 B1 Sample 1 Stack.czi', '20250307 B1 Sample 3 Stack.czi', '20250307 B1 Sample 2 Stack.czi']
Processing 20250307 B1 Sample 5 Stack.czi
Metadata: {'rgb': False, 'channel_axis': 2, 'translate': (0.0, 0.0, 0.0, 0.0), 'scale': (1.0, 0.5, 0.0974884033203125, 0.0974884033203125), 'contrast_limits': None, 'name': ['AF546-T1', 'DAPI-T2', 'AF647-T2']}
Image shape: (202, 3, 2048, 2048)
Loaded ../results/20250307 B1 P14 U34-B3-546 Chymotrypsin-B2-647 DAPI/20250307 B1 Sample 5 Stack_rois.npy
Loaded ../results/20250307 B1 P14 U34-B3-546 Chymotrypsin-B2-647 DAPI/20250307 B1 Sample 5 Stack_nuclei.csv
DAPI channel index: 1
Non-DAPI channels: [(0, 'AF546'), (2, 'AF647')]
Normalized image data using DAPI intensity.
Saving cell borders to ../results/20250307 B1 P14 U34-B3-546 Chymotrypsin-B2-647 DAPI/20250307 B1 Sample 5 Stack_cell_borders.npy